# 🏭 Theology AI Lab - Cloud Indexer

이 노트북은 **Google Drive**에 저장된 신학 문서(PDF/EPUB)를 **T4 GPU**를 사용하여 고속으로 인덱싱합니다.

### 🚀 사용법
1. **`inbox` 폴더**에 파일을 넣으세요.
2. 상단 메뉴의 **`런타임 > 모두 실행`**을 클릭하세요.
3. 완료되면 로컬 Streamlit 앱에서 검색하세요.

In [ ]:
# 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 2. 필수 라이브러리 설치 (약 1분 소요)
!pip install -q langchain chromadb sentence-transformers pymupdf pytesseract rank_bm25
!apt-get install -y tesseract-ocr > /dev/null

In [ ]:
# 3. 경로 설정 및 시스템 코드 로드
import sys
import os
from pathlib import Path

# Google Drive 내의 프로젝트 루트 경로 (사용자 환경에 맞게 수정 가능)
PROJECT_ROOT = Path("/content/drive/MyDrive/Theology_AI_LAB")
SYSTEM_PATH = PROJECT_ROOT / "03_System"

# 파이썬 경로에 추가하여 우리가 짠 모듈(processor_v4 등)을 불러올 수 있게 함
if str(SYSTEM_PATH) not in sys.path:
    sys.path.append(str(SYSTEM_PATH))

print(f"📂 프로젝트 루트: {PROJECT_ROOT}")
print(f"⚙️ 시스템 경로: {SYSTEM_PATH}")

# 경로 확인
if not SYSTEM_PATH.exists():
    raise FileNotFoundError(f"❌ 시스템 코드를 찾을 수 없습니다: {SYSTEM_PATH}\n'03_System' 폴더를 구글 드라이브 'Theology_AI_LAB' 안에 복사해주세요!")

In [ ]:
# 4. 인덱서 초기화 및 실행
from processor_v4 import ProjectV4Processor

# 경로 정의
inbox_dir = PROJECT_ROOT / "01_Library/inbox"
archive_dir = PROJECT_ROOT / "01_Library/archive"
db_dir = PROJECT_ROOT / "02_Brain/vector_db"

print("🚀 인덱싱 프로세서를 초기화합니다...")
# GPU 사용을 위해 device='cuda' 강제 설정 가능 (Processor 코드에 따라 자동 감지됨)
processor = ProjectV4Processor(db_dir=str(db_dir))

print("\n📥 Inbox 파일 스캔 중...")
# 인덱싱 실행
# 주의: Colab에서는 generator를 루프로 돌려야 출력이 보임
for update in processor.index_files_with_progress(inbox_dir, archive_dir):
    status = update.get("status")
    msg = update.get("message")
    
    if status == "indexing":
        print(f"[Processing] {msg}")
    elif status == "done":
        print(f"\n🎉 {msg}")
        stats = update.get("session_stats", {})
        print(f"   - 처리된 파일: {stats.get('processed_files', 0)}")
        print(f"   - 총 청크: {stats.get('total_chunks', 0)}")
    elif status == "error":
        print(f"❌ 오류 발생: {msg}")

print("\n✅ 모든 작업이 완료되었습니다. 로컬 앱에서 검색해보세요!")